In [1]:
import numpy as np
from tqdm import tqdm
from differential_evolution.DifferentialEvolution import DifferentialEvolution

NP = 50  # 个体数目
D = 10  # 目标函数中变量的个数
G = 8000  # 最大迭代数
CR = 0.1  # 交叉算子
Threshold = 1e-6  # 阈值
F = 0.5  # 变异算子
Left = -20  # 左边界
Right = 20  # 右边界

In [9]:
# 适应度函数/目标函数
def function(x):  # x的维度为10，也即D=10
    return sum(x ** 2)


# 求适应度
def fitness(x):
    result = np.empty(x.shape[1])  # 记录种群中个体的适应度
    # 计算每个个体的适应度
    for i in range(0, x.shape[1]):
        result[i] = function(x[:, i])
    return result

In [11]:


x = np.random.rand(D, NP) * (Right - Left) + Left  # 在[Left,Right]之间产生随机数

# 记录进化迭代的过程
number_generations = G
per_generation_optimal_value = np.zeros(G)

x_fitness = fitness(x)

de=DifferentialEvolution(NP, D, G, CR, Threshold, F, Left, Right)

with tqdm(total=G, desc="DE") as pbar:
    for i in range(0, G):
        #v=variation(x,F) # 变异
        v = de.variation_optimize(x)  # 变异的优化
        c = de.cross(x, v)  # 交叉

        c = de.boundary_process(c)  # 若超出边界则重新生成

        c_fitness = fitness(c)  # 对新的试验种群求适应度
        for m in range(0, x.shape[1]):
            if c_fitness[m] < x_fitness[m]:  # 贪婪地选择当前最优个体
                x[:, m] = c[:, m]
                x_fitness[m] = c_fitness[m]
        index = np.argmin(x_fitness)  # 记录最优个体索引
        per_generation_optimal_value[i] = x_fitness[index]  # 记录最优适应度
        # 动态更新进度条信息
        pbar.set_postfix({
            "当前迭代次数": i + 1,
            "最优个体索引": index,
            "目标函数最小值": x_fitness[index]
        })
        # 更新进度条
        pbar.update(1) 

DE: 100%|██████████| 8000/8000 [00:09<00:00, 858.03it/s, 当前迭代次数=8000, 最优个体索引=48, 目标函数最小值=3.33e-13]
